In [12]:

import datetime
import json
import threading
import time

from pandas import Timestamp
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [13]:
# load azure keys
import json
with open('../keys/azure.json') as json_file:
    azure_keys = json.load(json_file)
print(azure_keys.keys())


dict_keys(['stt_key1', 'stt_key2', 'openai_key1', 'openai_key2'])


In [14]:

def predict(context,question):
    print("-------------generating prediction-----------------")
    # data = json.loads(request.data)
    # data.context=data.context.replace('\n','\\n')
    # data.context=data.context.replace('\t','\\t')

    # context=request.args.get('context')
    # question=request.args.get('question')
    context=context.replace('\n','\\n')
    context=context.replace('\t','\\t')



    # input=f'paragraph: {context}'
    # instruction=f'Answer the following question from the paragraph: Question : {question}'
    # prompts=[[input,instruction]]
    prompt="""
    Answer the following question from the given paragraph: 
    #question start
    {question}
    #question end
    
    $paragraph start
    {context}
    #paragraph end
    """

    start_time = time.time()
    from openai import AzureOpenAI
    # from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    deployment = "red-gpt"
    endpoint="https://genai-warriors-openai-test.openai.azure.com/"
    # token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        # azure_ad_token_provider=token_provider,
        api_version="2024-02-01",
        api_key=azure_keys['openai_key1']
    )
    completion = client.completions.create(
        model=deployment,
        prompt=prompt,
        temperature=0.3,
        max_tokens=350,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    response=completion
    print(response)
    cfr=response.choices[0].content_filter_results
    pfr=response.prompt_filter_results
    res=response.choices[0].text
    pt=time.time() - start_time
    print(f"--------------time taken by generating response={pt} seconds")


    return  { 'output': f'{res}' }

In [15]:

def listen_msgs():
    coll_name='mhi_pred_app'
    user_uid='mhi_pred'
        
    # Use a service account.
    cred = credentials.Certificate(f'../keys/sa.json')

    if not firebase_admin._apps:
        app = firebase_admin.initialize_app(cred)

    db = firestore.client()
    
    # Create an Event for notifying main thread.
    callback_done = threading.Event()

    # Create a callback on_snapshot function to capture changes
    def on_snapshot(doc_snapshot, changes, read_time):
        for doc in doc_snapshot:
            print(f"Received document snapshot: {doc.id}")
            print(doc.to_dict())
            data=doc.to_dict()

            pred_res=predict(data['context'],data['message'])
            # pred_res={'output':"model response"}

            data['senderId']='chatbot@red'
            data['message']=pred_res['output']
            data['timestamp']=datetime.datetime.utcnow()
            print(f'sending response: '+data['message'])
            doc_id=str(round(time.time() * 1000))
            db.collection(coll_name).document(user_uid).collection('allMessages').document(doc_id).set(data)
            print(f"----------sent----------------with id: {doc_id} ")

        callback_done.set()

    doc_ref = db.collection(coll_name).document(user_uid).collection('userMessages').document("message")

    # Watch the document
    doc_watch = doc_ref.on_snapshot(on_snapshot)
    

    while True:
        print('', end='', flush=True)
        time.sleep(1)


In [16]:
try:
    print("listening for new messages-------------------")
    listen_msgs()
except Exception as e:
    print(f'error occured:\n\n\n {e}')  

listening for new messages-------------------


Received document snapshot: message
{'senderId': 'mhi@red', 'rType': '1', 'message': 'hi, is there any ai code agent which has memory of the shared code base to respond with the same structure code', 'context': 'hi, is there any ai code agent which has memory of the shared code base to respond with the same structure code\n', 'timestamp': DatetimeWithNanoseconds(2024, 5, 4, 3, 21, 34, 17000, tzinfo=datetime.timezone.utc), 'userNotificationToken': ''}
-------------generating prediction-----------------
Completion(id='cmpl-9OdirOWxZoK0FEKkUrch3haEKfJuc', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n    What is the context of the paragraph?\n    \n    The context of the paragraph is not specified.', content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1715659593, model

KeyboardInterrupt: 